In [17]:
import pandas as pd

df = pd.read_csv('data/train_pp.csv')
test_df = pd.read_csv('data/test_pp.csv')

X = df.drop(['accident_risk', 'id'], axis=1)
y = df['accident_risk']

In [36]:
def submission_generator(trained_model):
    test_df_preprocessed = test_df.drop('id', axis=1)
    return pd.concat([test_df['id'], pd.Series(trained_model.predict(test_df_preprocessed))], axis=1).rename({0: 'accident_risk'}, axis=1)

In [21]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error
import numpy as np 

### XGB

In [22]:
kf = KFold(5, shuffle=True)

def objective(trials):
    max_leaves = trials.suggest_int('max_leaves', 0, 256)
    max_depth = trials.suggest_int('max_depth', 2, 512)
    learning_rate = trials.suggest_float('learning_rate', 0.003, 0.3)
    n_estimators = trials.suggest_int('n_estimators', 24, 1024)

    clf = XGBRegressor(
        max_leaves=max_leaves,
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        device='cuda',
    )

    rmses = []
    for train_index, test_index in kf.split(X):
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]
        X_test = X.iloc[test_index]
        y_test = y.iloc[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        rmse = root_mean_squared_error(y_pred, y_test)
        rmses.append(rmse)

    return np.mean(rmses)

In [23]:
xgb_study = optuna.create_study()
xgb_study.optimize(objective, n_trials=50)

[I 2025-10-24 22:49:45,651] A new study created in memory with name: no-name-26cc668b-c6ff-4835-a2b9-82db7f6c987e
[I 2025-10-24 22:49:47,045] Trial 0 finished with value: 0.056054882006762494 and parameters: {'max_leaves': 119, 'max_depth': 449, 'learning_rate': 0.241918760799226, 'n_estimators': 62}. Best is trial 0 with value: 0.056054882006762494.
[W 2025-10-24 22:49:52,917] Trial 1 failed with parameters: {'max_leaves': 134, 'max_depth': 365, 'learning_rate': 0.13955529461031632, 'n_estimators': 989} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/woxane/anaconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_16606/1306643608.py", line 24, in objective
    clf.fit(X_train, y_train)
  File "/home/woxane/anaconda3/lib/python3.10/site-packages/xgboost/core.py", line 774, in inner_f
    return func(**kwargs)
  File "/home/woxane/anaconda3/lib

KeyboardInterrupt: 

In [37]:
clf_xgb = XGBRegressor(
    max_leaves= 76,
    max_depth=7,
    learning_rate=0.030293475264037178,
    n_estimators=721,
    device='cuda',
)
clf_xgb.fit(X, y)

submission = submission_generator(clf_xgb)
submission.to_csv('XGB_meta_optuna.csv', index=False)

In [13]:
xgb_study.best_params

{'max_leaves': 76,
 'max_depth': 7,
 'learning_rate': 0.030293475264037178,
 'n_estimators': 721}

### LGBM

In [38]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import root_mean_squared_error

In [39]:
kf = KFold(5, shuffle=True)

def objective(trials):
    num_leaves = trials.suggest_int('num_leaves', 4, 256)
    max_depth = trials.suggest_int('max_depth', -1, 512)
    learning_rate = trials.suggest_float('learning_rate', 0.003, 0.3)
    n_estimators = trials.suggest_int('n_estimatora', 24, 1024)

    clf = LGBMRegressor(
        num_leaves=num_leaves,
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
    )

    rmses = []
    for train_index, test_index in kf.split(X):
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]
        X_test = X.iloc[test_index]
        y_test = y.iloc[test_index]

        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        rmse = root_mean_squared_error(y_pred, y_test)
        rmses.append(rmse)

    return np.mean(rmses)

In [40]:
lgbm_study = optuna.create_study()
lgbm_study.optimize(objective, n_trials=100)

[I 2025-10-24 22:54:24,310] A new study created in memory with name: no-name-ca861104-d8f1-41d9-bf5b-43947486f64b


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003310 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 424
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352302
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003340 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352331
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003715 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

[I 2025-10-24 22:54:28,803] Trial 0 finished with value: 0.05599873452016478 and parameters: {'num_leaves': 10, 'max_depth': 171, 'learning_rate': 0.1766503739314514, 'n_estimatora': 706}. Best is trial 0 with value: 0.05599873452016478.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004656 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 428
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352493
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002927 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352290
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004246 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

[I 2025-10-24 22:54:36,046] Trial 1 finished with value: 0.05664223748359115 and parameters: {'num_leaves': 84, 'max_depth': 336, 'learning_rate': 0.21862366771258807, 'n_estimatora': 656}. Best is trial 0 with value: 0.05599873452016478.


...

[I 2025-10-24 23:07:21,539] Trial 97 finished with value: 0.056152632977101394 and parameters: {'num_leaves': 34, 'max_depth': 18, 'learning_rate': 0.17281835011421876, 'n_estimatora': 762}. Best is trial 92 with value: 0.05598629831711309.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002374 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 429
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002953 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 422
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352188
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004170 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

[I 2025-10-24 23:07:26,864] Trial 98 finished with value: 0.05600262479684305 and parameters: {'num_leaves': 19, 'max_depth': 73, 'learning_rate': 0.12440830915408377, 'n_estimatora': 639}. Best is trial 92 with value: 0.05598629831711309.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002912 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 428
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352547
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004424 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 431
[LightGBM] [Info] Number of data points in the train set: 414203, number of used features: 17
[LightGBM] [Info] Start training from score 0.352444
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004434 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enoug

[I 2025-10-24 23:07:31,439] Trial 99 finished with value: 0.056051297840909696 and parameters: {'num_leaves': 7, 'max_depth': 70, 'learning_rate': 0.11971961519125265, 'n_estimatora': 643}. Best is trial 92 with value: 0.05598629831711309.


In [44]:
clf_lgbm = LGBMRegressor(**lgbm_study.best_params)
clf_lgbm.fit(X, y)

submission = submission_generator(clf_lgbm)
submission.to_csv('LGBM_meta_optuna.csv', index=False)

[LightGBM] [Warning] Unknown parameter: n_estimatora
[LightGBM] [Warning] Unknown parameter: n_estimatora
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006544 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 432
[LightGBM] [Info] Number of data points in the train set: 517754, number of used features: 17
[LightGBM] [Info] Start training from score 0.352377
[LightGBM] [Warning] Unknown parameter: n_estimatora


In [42]:
lgbm_study.best_params

{'num_leaves': 42,
 'max_depth': 61,
 'learning_rate': 0.03813337721131469,
 'n_estimatora': 807}